In [1]:
import os
import joblib

import pandas as pd

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from sklearn.metrics import classification_report

In [2]:
import numpy as np

input_file = 'D:/dataset/viscom/TA/data.csv'
df = pd.read_csv(input_file)

df

,WRIST_X,WRIST_Y,WRIST_Z,THUMB_CMC_X,THUMB_CMC_Y,THUMB_CMC_Z,THUMB_MCP_X,THUMB_MCP_Y,THUMB_MCP_Z,THUMB_IP_X,...,PINKY_PIP_Y,PINKY_PIP_Z,PINKY_DIP_X,PINKY_DIP_Y,PINKY_DIP_Z,PINKY_TIP_X,PINKY_TIP_Y,PINKY_TIP_Z,NUMBER,LETTER
0,0.571034,0.662065,-7.807466e-07,0.473849,0.583998,-0.005987,0.392715,0.467240,-0.025805,0.321770,...,0.230389,-0.135626,0.546443,0.177951,-0.157419,0.474000,0.174955,-0.163806,0.0,NaN
1,0.482472,0.647370,-5.996167e-07,0.342938,0.588442,0.006532,0.232317,0.447858,-0.008136,0.174120,...,0.194326,-0.196713,0.289174,0.168394,-0.208204,0.213607,0.189604,-0.207796,0.0,NaN
2,0.380521,0.266158,-2.160466e-07,0.367195,0.272093,-0.011354,0.332329,0.278134,-0.016402,0.306362,...,0.328112,-0.020859,0.209677,0.357561,-0.022637,0.205147,0.385581,-0.022078,0.0,NaN
3,0.433416,0.819656,-9.284144e-07,0.331050,0.744098,-0.000364,0.256925,0.644873,-0.009013,0.215765,...,0.405161,-0.075169,0.371768,0.368570,-0.085466,0.317381,0.371379,-0.088116,0.0,NaN
4,0.410049,0.763829,-9.258480e-07,0.305733,0.702219,-0.011086,0.228612,0.608595,-0.024548,0.187016,...,0.371528,-0.077441,0.362513,0.336863,-0.090728,0.304557,0.342354,-0.093968,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64210,0.633891,0.884113,-9.847142e-08,0.596272,0.852404,-0.046660,0.569814,0.791661,-0.080595,0.575327,...,0.805794,-0.104452,0.679830,0.846776,-0.103194,0.669114,0.831234,-0.095521,NaN,Z
64211,0.629664,0.881855,-1.936707e-07,0.594472,0.849347,-0.037089,0.570097,0.787039,-0.063848,0.575564,...,0.803748,-0.089654,0.680367,0.840722,-0.087647,0.670151,0.827668,-0.080729,NaN,Z
64212,0.632679,0.882121,-1.441144e-07,0.595479,0.849925,-0.044451,0.569643,0.788898,-0.076009,0.575442,...,0.805685,-0.101197,0.676469,0.846899,-0.100586,0.665062,0.832417,-0.093405,NaN,Z
64213,0.629037,0.884366,-2.504402e-07,0.593912,0.850297,-0.035675,0.568009,0.787228,-0.060410,0.575331,...,0.800616,-0.085155,0.677656,0.838333,-0.084610,0.665922,0.826095,-0.078752,NaN,Z


In [3]:
# Combine 'LETTER' and 'NUMBER' into a single column
df['TARGET'] = df['LETTER'].fillna(df['NUMBER'])

# Convert the new target to a suitable format for classification
df['TARGET'] = df['TARGET'].astype(str)

# Now you can split your data
X = df.drop(['LETTER', 'NUMBER', 'TARGET'], axis=1)
y = df['TARGET']

In [4]:
y

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
64210      Z
64211      Z
64212      Z
64213      Z
64214      Z
Name: TARGET, Length: 64215, dtype: object

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(
    X, y,
    test_size=0.25,
    random_state=1337
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=0.25,
    random_state=1337
)

In [6]:
cat_clf = CatBoostClassifier(
    iterations=1000,  # reduce the number of iterations
    depth=10,  # set the depth of the trees
    learning_rate=0.1,  # set the learning rate
    loss_function='MultiClass',  # keep the loss function as 'MultiClass'
    l2_leaf_reg=3,  # set the L2 regularization term
    task_type='CPU'  # keep the task type as 'CPU'
)

cat_clf.fit(
    X=X_train, y=y_train,
    eval_set=(X_dev, y_dev),
    early_stopping_rounds=100,
    plot=True
)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.8481592	test: 1.8302265	best: 1.8302265 (0)	total: 11.4s	remaining: 3h 10m 38s
1:	learn: 1.6067381	test: 1.5931941	best: 1.5931941 (1)	total: 22.3s	remaining: 3h 5m 21s
2:	learn: 1.4388237	test: 1.4296204	best: 1.4296204 (2)	total: 33s	remaining: 3h 2m 35s
3:	learn: 1.2956823	test: 1.2899116	best: 1.2899116 (3)	total: 43.7s	remaining: 3h 1m 15s
4:	learn: 1.1868035	test: 1.1846988	best: 1.1846988 (4)	total: 54.4s	remaining: 3h 31s
5:	learn: 1.0937391	test: 1.0937996	best: 1.0937996 (5)	total: 1m 4s	remaining: 2h 59m 20s
6:	learn: 1.0068530	test: 1.0088207	best: 1.0088207 (6)	total: 1m 15s	remaining: 2h 58m 35s
7:	learn: 0.9348794	test: 0.9379030	best: 0.9379030 (7)	total: 1m 23s	remaining: 2h 52m
8:	learn: 0.8744879	test: 0.8798566	best: 0.8798566 (8)	total: 1m 29s	remaining: 2h 45m 4s
9:	learn: 0.8183805	test: 0.8261754	best: 0.8261754 (9)	total: 1m 36s	remaining: 2h 39m 51s
10:	learn: 0.7693736	test: 0.7791034	best: 0.7791034 (10)	total: 1m 43s	remaining: 2h 35m 25s
11:	le

In [10]:
y_pred = cat_clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.85      0.80      0.83        41
         1.0       0.96      0.96      0.96       113
         2.0       0.94      0.90      0.92        83
         3.0       0.99      0.98      0.98       171
         4.0       0.92      0.93      0.92        84
         5.0       0.98      1.00      0.99       218
         6.0       0.95      0.93      0.94       107
         7.0       0.98      0.96      0.97       153
         8.0       0.96      0.96      0.96       144
         9.0       0.98      0.99      0.99       102
           A       0.96      0.98      0.97        94
           B       0.97      0.97      0.97        92
           C       0.98      0.96      0.97        54
           D       1.00      0.97      0.99        68
           E       0.96      0.94      0.95       106
           F       1.00      0.99      0.99        79
           G       0.97      0.94      0.95        65
           H       0.97    

In [11]:
output_dir = os.path.dirname(input_file)
output_model_path = os.path.join(output_dir, "asl_alphabet_CatBoostClassifier.pkl")

os.makedirs(output_dir, exist_ok=True)
joblib.dump(cat_clf, output_model_path, compress=9)

['D:/dataset/viscom/TA\\asl_alphabet_CatBoostClassifier.pkl']

In [22]:
from sklearn.metrics import confusion_matrix
import pandas as pd

# y_pred is already defined in your code
# y_pred = cat_clf.predict(X_test)

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Get unique class labels
class_labels = np.unique(y_test)

# Convert the confusion matrix to a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)


# Set the display option to show all rows
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(cm_df)

     0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0   A   B   C   D    E  \
0.0   33    0    0    0    0    0    0    0    0    0   1   0   0   0    1   
1.0    1  108    1    0    0    0    0    1    0    0   0   0   0   0    0   
2.0    0    4   75    0    0    0    0    0    0    0   0   0   0   0    0   
3.0    0    0    0  167    0    0    0    0    2    0   0   0   0   0    0   
4.0    0    0    0    0   78    2    0    1    0    0   0   1   0   0    0   
5.0    0    0    0    0    0  218    0    0    0    0   0   0   0   0    0   
6.0    0    0    3    0    1    0   99    0    0    0   0   1   0   0    0   
7.0    0    0    0    0    1    1    0  147    4    0   0   0   0   0    0   
8.0    0    0    0    0    3    1    0    1  138    1   0   0   0   0    0   
9.0    0    0    0    0    0    0    0    0    0  101   0   0   0   0    0   
A      0    0    0    0    0    0    0    0    0    0  92   0   0   0    0   
B      0    0    0    0    2    0    0    0    0    0   0  89   